# Image Captioning project
Build our first prototype model for Image captioning using structure combination of CNN and Transformer.

In [19]:
name_1 = 'Qinghuan Liu'
name_2 = 'Jingkai Zhou'
name_3 = 'Chang Li'
naem_4 = 'Yawen Liu'
planning_group = '28'

---
## 0. Import
import possible files here:

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


---
## 1. Load dataset
In this part, load and preprocess famous captioning dataset ['Flickr 8k Dataset'](https://www.kaggle.com/datasets/adityajn105/flickr8k). 

---
### 1.1 Load dataset from disk and split into training / validation / test sets
Since the size of our trainning archive is not relatively large, the ratio is chosen according to the technique [Data splitting technique to fit any Machine Learning Model](https://towardsdatascience.com/data-splitting-technique-to-fit-any-machine-learning-model-c0d7f3f1c790)

In [21]:
from pathlib import Path
# Some file system operation are not covered by 'Path' and we use 'shutil' for that
import shutil

# Regular expressions are used to find patterns in strings
import re

# For splitting the data
from sklearn.model_selection import train_test_split





Statistical information on dataset:

In [22]:
# Path to Flickr8K_ photos
path_Flickr_jpg = "G:/GitHub/Deep_Learning_Proj/archive/Images"
# Path to caption file
path_Flickr_text = "G:/GitHub/Deep_Learning_Proj/archive/captions.txt"

image_all = Path.cwd() / "G:/GitHub/Deep_Learning_Proj/archive/Images/"

all_image_filenames = list(image_all.glob("*.jpg"))

print(len(all_image_filenames))
print(all_image_filenames[0])

8091
G:\GitHub\Deep_Learning_Proj\archive\Images\1000268201_693b08cb0e.jpg


split data with predefined ratio

In [23]:
split_ratio_dataset = 0.2

image_train, image_val = \
train_test_split(all_image_filenames,  
              test_size = split_ratio_dataset,
              random_state = 2)

# Following operations create train and val dataset folders located at the same root as this proj.ipynb
subdirectories = {"./image_train": image_train,
                 "./image_val": image_val
                 }

for subdirectory in subdirectories.keys():
    subdirectory = Path(subdirectory)
    subdirectory.mkdir(parents=True, exist_ok=True)
    
    
# Put the training and validation data in the respective folders
def fill_sub_dir(sub_dir, file_subset):
    """This function copies files from the `train_all` to a `<sub_dir>`
    A more efficient solution would be to use "symbolic links" (see https://kb.iu.edu/d/abbe)
    but for simplicity hard copies is used instead.
    """
    for file in file_subset:
        file_path = Path.cwd() / sub_dir / file.name
        shutil.copyfile(file, file_path)
        
for sub_dir, file_subset in subdirectories.items():
    fill_sub_dir(sub_dir, file_subset)


In [24]:
print(len(image_train))
print(len(image_val))

6472
1619


Point to training and validation path.

In [25]:
train_path = "./image_train/"
val_path = "./image_val/"
caption_path = "./captions.txt"

Create a dictionary to store image names and its corresponding five captions {Key:Value} = {'img_name': \['cap1', 'cap2', 'cap3', 'cap4', 'cap5' \]}

In [27]:
with open('captions.txt') as f:
    lines = f.readlines()

lines.pop(0) # remove first desciption line in lines
    
dict_img_caps =  {}
for i in range(int(len(lines)/5)):
    txt_0 = lines[5*i+0].split(",")
    txt_1 = lines[5*i+1].split(",")
    txt_2 = lines[5*i+2].split(",")
    txt_3 = lines[5*i+3].split(",")
    txt_4 = lines[5*i+4].split(",")
    
    img_name = txt_0[0] # get img file name
    txt_cap = [txt_0[1], txt_1[1], txt_2[1], txt_3[1], txt_4[1]] # list of five captions per img
    
    dict_img_caps[img_name] = txt_cap



## 1.2 Create image class
Here, as what we did before in HA1, new class img_cap is created with some neccessary functions.

In [ ]:
from torch.utils.data import Dataset
from itertools import chain
from PIL import Image
from pathlib import Path

class img_cap(Dataset):
    
    def __init__(self, root, transform):
        """Constructor
        
        Args:
            root (Path/str): Filepath to the data root, e.g. './small_train'
            transform (Compose): A composition of image transforms, see below.
        """
        
        root = Path(root)
        if not (root.exists() and root.is_dir()):
            raise ValueError(f"Data root '{root}' is invalid")
            
        self.root = root
        self.transform = transform
        self._dog_label = dog_label
        self._cat_label = cat_label
        
        # Collect samples, both cat and dog and store pairs of (filepath, label) in a simple list.
        self._samples = self._collect_samples()
            
    def __getitem__(self, index):
        """Get sample by index
        
        Args:
            index (int)
        
        Returns:
             The index'th sample (Tensor, int)
        """
        # Access the stored path and label for the correct index
        path, label = self._samples[index]
        # Load the image into memory
        img = Image.open(path)
        # Perform transforms, if any.
        if self.transform is not None:
            img = self.transform(img)
        return img, label
    
    def __len__(self):
        """Total number of samples"""
        # YOUR CODE HERE
        return len(self._samples)
    
    def _collect_samples(self):
        """Collect all paths and labels
        
        Helper method for the constructor
        """
        # Iterator over dog filpath
        dog_paths = self._collect_imgs_sub_dir(self.root / "dogs")
        # Iterator of pairs (path, dog label)
        # Again, we use the `map` function to create an iterator. It's use is not as common as the so called
        # 'list comprehension' you've previously seen, but a good alternative to have seen.
        dog_paths_and_labels = map(lambda path: (path, self._dog_label), dog_paths)
        # Same for cats
        cat_paths = self._collect_imgs_sub_dir(self.root / "cats")
        cat_paths_and_labels = map(lambda path: (path, self._cat_label), cat_paths)
        # Sorting is not strictly necessary, but filesystem globbing (wildcard search) is not deterministic,
        # and consistency is nice when debugging.
        return sorted(list(chain(dog_paths_and_labels, cat_paths_and_labels)), key=lambda x: x[0].stem)
     
    @staticmethod
    def _collect_imgs_sub_dir(sub_dir: Path):
        """Collect image paths in a directory
        
        Helper method for the constructor
        """
        if not sub_dir.exists():
            raise ValueError(f"Data root '{self.root}' must contain sub dir '{sub_dir.name}'")
        return sub_dir.glob("*.jpg")
    
    def get_sample_by_id(self, id_):
        """Get sample by image id
        
        Convenience method for exploration.
        The indices does not correspond to the image id's in the filenames.
        Here is a (rather inefficient) way of inspecting a specific image.
        
        Args:
            id_ (str): Image id, e.g. `dog.321`
        """
        id_index = [path.stem for (path, _) in self._samples].index(id_)
        return self[id_index]
